# 1. 利用梯度磁带计算导数

In [4]:
import tensorflow as tf
import numpy as np

# 计算函数 f(x) = a*x**2 + b*x + c 在x=2.0处的导数

x = tf.Variable(2.0, name='x', dtype=tf.float32)
a = tf.constant(1.0)
b = tf.constant(-2.0)
c = tf.constant(1.0)

with tf.GradientTape() as tape:
    y = a*tf.pow(x, 2) + b*x + c
    
dy_dx = tape.gradient(y, x)
print(dy_dx)

tf.Tensor(2.0, shape=(), dtype=float32)
